# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [46]:
import sys
from datetime import datetime as dt
from typing import Callable

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [ ]:
output = open("./some_test_data.txt", 'w')

In [ ]:
output.write('123')

In [ ]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [7]:
import sys
from datetime import datetime as dt
from typing import Union

original_write = sys.stdout.write


# Task 1

def my_write(string_text: str) -> Union[str, int]:
    """
    Дописывает к тексту текущие дату и время
    """
    string_text_with_time = f'{dt.now().strftime("[%Y-%m-%d %H:%M:%S]:")} {string_text}'

    if string_text == '\n':
        return original_write('\n')
    else:
        return original_write(string_text_with_time)


if __name__ == '__main__':
    sys.stdout.write = my_write
    print('1, 2, 3')
    sys.stdout.write = original_write
    print('1, 2, 3')

[2022-12-09 18:52:26]: 1, 2, 3
1, 2, 3


Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [6]:
import sys
from typing import Callable
# from decorators_hw_task_1 import my_write, original_write


# Task 2

def timed_output(function: Callable) -> Callable:
    def wrapper(*args, **kwargs):
        sys.stdout.write = my_write
        function_res = function(*args, **kwargs)
        sys.stdout.write = original_write

        return function_res

    return wrapper


@timed_output
def s(*args, **kwargs):
    return 22


@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')


if __name__ == '__main__':
    print(s(1, 2, 3, "aaa", True, name=11))
    print_greeting("Nikita")

22
[2022-12-09 18:52:21]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [5]:
import sys
from typing import Callable


# Task 3
def redirect_output(filepath: str) -> Callable:
    def wrapper_1(func: Callable) -> Callable:
        def wrapper_2(*args, **kwargs):
            with open(filepath, 'w') as file:
                original_stdout = sys.stdout
                sys.stdout = file
                func(*args, **kwargs)
                sys.stdout = original_stdout

        return wrapper_2

    return wrapper_1


@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()


@redirect_output('./function_output_2.txt')
def some_other_function(*args, **kwargs):
    res1 = sum(args)
    res2 = 7
    print(res1 ** res2)


if __name__ == '__main__':
    calculate()
    with open('./function_output.txt', 'r') as ffile:
        print(ffile.read())
    some_other_function(1, 2, 3, x="AAA")
    with open('./function_output_2.txt', 'r') as ffile:
        print(ffile.read())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 

279936

